参考：https://blog.csdn.net/baoyan2015/article/details/145497813

https://chat.baidu.com/search?isShowHello=1&pd=csaitab&setype=csaitab&extParamsJson=%7B%22enter_type%22%3A%22ai_explore_home%22%7D&usedModel=%7B%22modelName%22%3A%22DeepSeek-R1%22%7D

MLA只是，将原始的 Linear映射矩阵需要存储 768*768的参数量变为，768*rank + rank*768，将需要存储的参数量变少
但是q-k需要计算的多头注意力并没有变化

In [ ]:
class MLA(nn.Module):
    def __init__(self,nums_head):
        super(MLA, self).__init__()
        self.nums_head = nums_head
        self.wka = nn.Linear(token_dim, lora_k_rank)
        self.wkb = nn.Linear(lora_k_rank, nums_head*(q_head_dim + q_rp_dim))
        self.wkva = nn.Linear(token_dim, lora_kv_rank + k_rp_dim)
        self.wkvb = nn.Linear(lora_kv_rank, nums_head*(k_head_dim+v_head_dim))
        self.olinear = nn.Linear(token_dim, token_dim)

    def forward(self, mask):
        batch_size, seq_len = x.size()[:2]
        q_rp = self.wqb(self.wqa(x))
        q_rp = q_rp.view(batch_size,seq_len, self.nums_head, (q_head_dim + q_rp_dim))
        q, rp = torch.split(q_rp, [q_head_dim, q_rp_dim],dim=-1)
        rp = apply_rope(rp)
        q = torch.cat([q,rp],dim = -1)

        kv_rk = self.wkva(x)
        kv, rk = torch.split(kv_rk, [low_rk_rank, k_rp_dim],dim=-1)
        rk = apply_rope(rk)
        kv = self.wkvb(rk)
        kv = kv.view(batch_size, seq_len, nums_head, k_head_dim + v_head_dim)
        k,v = torch.split(kv, [k_head_dim, v_head_dim], dim = -1)
        k = torch.cat([k,rk],dim = -1)

        attn = torch.matul(q.transpose(-1,-2),k) // torch.sqrt(torch.tensor(q_head_dim + q_rp_dim))

        if mask:
            attn += mask*-1e-9

        attp = torch.softmax(attn,dim = -1)
        attns = torch.matul(attp, v).transpose(1,2).contiguous().view(batch_size,seq_len,num_heads*v_head_dim)
        output = self.olinear(attns)
        return output

In [ ]:
## DeepSeek MLA源码
class MLA(nn.Module):
    """
    Multi-Headed Attention Layer (MLA).

    Attributes:
        dim (int): Dimensionality of the input features.
        n_heads (int): Number of attention heads.
        n_local_heads (int): Number of local attention heads for distributed systems.
        q_lora_rank (int): Rank for low-rank query projection.
        kv_lora_rank (int): Rank for low-rank key/value projection.
        qk_nope_head_dim (int): Dimensionality of non-positional query/key projections.
        qk_rope_head_dim (int): Dimensionality of rotary-positional query/key projections.
        qk_head_dim (int): Total dimensionality of query/key projections.
        v_head_dim (int): Dimensionality of value projections.
        softmax_scale (float): Scaling factor for softmax in attention computation.
    """
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.dim = args.dim  # 输入特征的维度d
        self.n_heads = args.n_heads # 128
        self.n_local_heads = args.n_heads // world_size  # word_size = 1  进程数
        self.q_lora_rank = args.q_lora_rank  # 低秩查询投影的秩   0表示不使用低秩 1536
        self.kv_lora_rank = args.kv_lora_rank # 低秩键/值投影的秩 512
        self.qk_nope_head_dim = args.qk_nope_head_dim # 128
        self.qk_rope_head_dim = args.qk_rope_head_dim # 64
        self.qk_head_dim = args.qk_nope_head_dim + args.qk_rope_head_dim  # 128+64
        self.v_head_dim = args.v_head_dim # 128

        if self.q_lora_rank == 0:
            self.wq = ColumnParallelLinear(self.dim, self.n_heads * self.qk_head_dim)
        else:
            self.wq_a = Linear(self.dim, self.q_lora_rank) # q_lora_rank = 1536   7168*1536
            self.q_norm = RMSNorm(self.q_lora_rank) # 1536
            self.wq_b = ColumnParallelLinear(self.q_lora_rank, self.n_heads * self.qk_head_dim) # 1536 * 128*(128+64)
        self.wkv_a = Linear(self.dim, self.kv_lora_rank + self.qk_rope_head_dim) # 7168*(512+64)
        self.kv_norm = RMSNorm(self.kv_lora_rank)  # 512
        self.wkv_b = ColumnParallelLinear(self.kv_lora_rank, self.n_heads * (self.qk_nope_head_dim + self.v_head_dim)) # 512 * 128*(128+128)
        self.wo = RowParallelLinear(self.n_heads * self.v_head_dim, self.dim)  # 128*128 * 7168
        self.softmax_scale = self.qk_head_dim ** -0.5
        if args.max_seq_len > args.original_seq_len:
            mscale = 0.1 * args.mscale * math.log(args.rope_factor) + 1.0
            self.softmax_scale = self.softmax_scale * mscale * mscale

        if attn_impl == "naive":
            self.register_buffer("k_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.n_local_heads, self.qk_head_dim), persistent=False)
            self.register_buffer("v_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.n_local_heads, self.v_head_dim), persistent=False)
        else:
            self.register_buffer("kv_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.kv_lora_rank), persistent=False)
            self.register_buffer("pe_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.qk_rope_head_dim), persistent=False)

    def forward(self, x: torch.Tensor, start_pos: int, freqs_cis: torch.Tensor, mask: Optional[torch.Tensor]):
        """
        Forward pass for the Multi-Headed Attention Layer (MLA).

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_len, dim).
            start_pos (int): Starting position in the sequence for caching.
            freqs_cis (torch.Tensor): Precomputed complex exponential values for rotary embeddings.
            mask (Optional[torch.Tensor]): Mask tensor to exclude certain positions from attention.

        Returns:
            torch.Tensor: Output tensor with the same shape as the input.
        """
        bsz, seqlen, _ = x.size()  # batch_size, seq_len, dim   1，2，7168
        end_pos = start_pos + seqlen
        if self.q_lora_rank == 0:
            q = self.wq(x)
        else:
            q = self.wq_b(self.q_norm(self.wq_a(x)))  #  1，2，7168 -->  1，2，1536  --> 1，2，128*(128+64)  先降维，再升维
        q = q.view(bsz, seqlen, self.n_local_heads, self.qk_head_dim)  # 1，2，128，128+64
        q_nope, q_pe = torch.split(q, [self.qk_nope_head_dim, self.qk_rope_head_dim], dim=-1) # 对最后一个维度进行切分 1，2，128，128 && 1，2，128，64
        q_pe = apply_rotary_emb(q_pe, freqs_cis)
        kv = self.wkv_a(x)  # 1，2，7168 --> 1，2，512+64  （） # 直接降维
        kv, k_pe = torch.split(kv, [self.kv_lora_rank, self.qk_rope_head_dim], dim=-1)  # 1，2，512 && 1，2，64
        k_pe = apply_rotary_emb(k_pe.unsqueeze(2), freqs_cis) # 1，2，1，64
        if attn_impl == "naive":
            q = torch.cat([q_nope, q_pe], dim=-1) # 1，2，128，128+64
            kv = self.wkv_b(self.kv_norm(kv)) # 1，2，512 --> 1，2，128*(128+128)  对kv进行生维
            kv = kv.view(bsz, seqlen, self.n_local_heads, self.qk_nope_head_dim + self.v_head_dim)  # 1，2，128，128+128（516）
            k_nope, v = torch.split(kv, [self.qk_nope_head_dim, self.v_head_dim], dim=-1) # 1，2，128，128 && 1，2，128，128
            k = torch.cat([k_nope, k_pe.expand(-1, -1, self.n_local_heads, -1)], dim=-1) # 1，2，128，128 && 1，2，1，64  --> 1，2，128，128+64
            self.k_cache[:bsz, start_pos:end_pos] = k
            self.v_cache[:bsz, start_pos:end_pos] = v
            scores = torch.einsum("bshd,bthd->bsht", q, self.k_cache[:bsz, :end_pos]) * self.softmax_scale  # 1，2，128，128  * 1，2，128，128+64
        else:
            wkv_b = self.wkv_b.weight if self.wkv_b.scale is None else weight_dequant(self.wkv_b.weight, self.wkv_b.scale, block_size) # 512 * 128*(128+128)
            wkv_b = wkv_b.view(self.n_local_heads, -1, self.kv_lora_rank) # 128, 128+128, 512
            q_nope = torch.einsum("bshd,hdc->bshc", q_nope, wkv_b[:, :self.qk_nope_head_dim]) # 1，2，128，128
            self.kv_cache[:bsz, start_pos:end_pos] = self.kv_norm(kv) # 1，2，512 --> 1，2，512
            self.pe_cache[:bsz, start_pos:end_pos] = k_pe.squeeze(2) # 1，2，1，64 --> 1，2，64
            scores = (torch.einsum("bshc,btc->bsht", q_nope, self.kv_cache[:bsz, :end_pos]) +
                      torch.einsum("bshr,btr->bsht", q_pe, self.pe_cache[:bsz, :end_pos])) * self.softmax_scale # 1，2，128，128  * 1，2，128，128+64
        if mask is not None:
            scores += mask.unsqueeze(1)
        scores = scores.softmax(dim=-1, dtype=torch.float32).type_as(x)
        if attn_impl == "naive":
            x = torch.einsum("bsht,bthd->bshd", scores, self.v_cache[:bsz, :end_pos]) # 1，2，128，128
        else:
            x = torch.einsum("bsht,btc->bshc", scores, self.kv_cache[:bsz, :end_pos])
            x = torch.einsum("bshc,hdc->bshd", x, wkv_b[:, -self.v_head_dim:])
        x = self.wo(x.flatten(2)) # 1，2，128，128 --> 1，2，7168
        return x

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadLatentAttention(nn.Module):
    def __init__(self, d_model, num_heads, latent_rank=32):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.latent_rank = latent_rank
        self.head_dim = d_model // num_heads

        # 低秩分解参数：键（Key）和值（Value）的分解矩阵 (原理2.1)
        self.U_k = nn.Linear(d_model, latent_rank * num_heads, bias=False)
        self.V_k = nn.Linear(latent_rank, self.head_dim, bias=False)
        self.U_v = nn.Linear(d_model, latent_rank * num_heads, bias=False)
        self.V_v = nn.Linear(latent_rank, self.head_dim, bias=False)

        # 查询（Query）的常规投影
        self.W_q = nn.Linear(d_model, d_model, bias=False)

        # 动态融合门控权重生成 (原理2.3)
        self.gate = nn.Linear(d_model, num_heads)

        # 潜在偏置参数 (原理2.2中的L_i)
        self.latent_bias = nn.Parameter(torch.zeros(num_heads, 1, 1))

        # 输出投影
        self.out_proj = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape

        # --- 低秩键/值投影 (原理2.1) ---
        # 键投影：X -> U_k -> V_k
        k_latent = self.U_k(x)  # [B, S, latent_rank * H]
        k_latent = k_latent.view(batch_size, seq_len, self.num_heads, self.latent_rank)
        k = self.V_k(k_latent)  # [B, S, H, D/H]

        # 值投影：X -> U_v -> V_v
        v_latent = self.U_v(x)
        v_latent = v_latent.view(batch_size, seq_len, self.num_heads, self.latent_rank)
        v = self.V_v(v_latent)

        # --- 查询投影 ---
        q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.head_dim)

        # 维度调整 (多头拆分)
        q = q.permute(0, 2, 1, 3)  # [B, H, S, D/H]
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        # --- 注意力计算 (原理2.2) ---
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_dim))
        attn_scores = attn_scores + self.latent_bias  # 添加潜在偏置L_i

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_output = torch.matmul(attn_weights, v)  # [B, H, S, D/H]

        # --- 动态头融合 (原理2.3) ---
        # 生成门控权重g_i
        gate_weights = F.softmax(self.gate(x), dim=-1)  # [B, S, H]
        gate_weights = gate_weights.permute(0, 2, 1).unsqueeze(-1)  # [B, H, S, 1]

        # 加权融合
        attn_output = attn_output * gate_weights
        attn_output = attn_output.sum(dim=1)  # [B, S, D/H]
        attn_output = attn_output.reshape(batch_size, seq_len, self.d_model)

        # 输出投影
        return self.out_proj(attn_output)

# ---------- 使用示例 ----------
if __name__ == "__main__":
    d_model = 512
    num_heads = 8
    seq_len = 128
    batch_size = 4

    mla = MultiHeadLatentAttention(d_model, num_heads, latent_rank=32)
    x = torch.randn(batch_size, seq_len, d_model)
    output = mla(x)
    print(output.shape)  # [4, 128, 512]


RuntimeError: shape '[4, 128, 512]' is invalid for input of size 32768

以下是基于PyTorch框架的MLA（Multi-head Latent Attention）简化版代码实现，包含‌低秩压缩‌和‌动态头融合‌核心逻辑。代码注释中标注了与原理对应的关键步骤。

MLA 完整代码实现‌

```pycon
python
Copy Code
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadLatentAttention(nn.Module):
    def __init__(self, d_model, num_heads, latent_rank=32):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.latent_rank = latent_rank
        self.head_dim = d_model // num_heads

        # 低秩分解参数：键（Key）和值（Value）的分解矩阵 (原理2.1)
        self.U_k = nn.Linear(d_model, latent_rank * num_heads, bias=False)
        self.V_k = nn.Linear(latent_rank, self.head_dim, bias=False)
        self.U_v = nn.Linear(d_model, latent_rank * num_heads, bias=False)
        self.V_v = nn.Linear(latent_rank, self.head_dim, bias=False)

        # 查询（Query）的常规投影
        self.W_q = nn.Linear(d_model, d_model, bias=False)

        # 动态融合门控权重生成 (原理2.3)
        self.gate = nn.Linear(d_model, num_heads)

        # 潜在偏置参数 (原理2.2中的L_i)
        self.latent_bias = nn.Parameter(torch.zeros(num_heads, 1, 1))

        # 输出投影
        self.out_proj = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape

        # --- 低秩键/值投影 (原理2.1) ---
        # 键投影：X -> U_k -> V_k
        k_latent = self.U_k(x)  # [B, S, latent_rank * H]
        k_latent = k_latent.view(batch_size, seq_len, self.num_heads, self.latent_rank)
        k = self.V_k(k_latent)  # [B, S, H, D/H]

        # 值投影：X -> U_v -> V_v
        v_latent = self.U_v(x)
        v_latent = v_latent.view(batch_size, seq_len, self.num_heads, self.latent_rank)
        v = self.V_v(v_latent)

        # --- 查询投影 ---
        q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.head_dim)

        # 维度调整 (多头拆分)
        q = q.permute(0, 2, 1, 3)  # [B, H, S, D/H]
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        # --- 注意力计算 (原理2.2) ---
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_dim))
        attn_scores = attn_scores + self.latent_bias  # 添加潜在偏置L_i

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_output = torch.matmul(attn_weights, v)  # [B, H, S, D/H]

        # --- 动态头融合 (原理2.3) ---
        # 生成门控权重g_i
        gate_weights = F.softmax(self.gate(x), dim=-1)  # [B, S, H]
        gate_weights = gate_weights.permute(0, 2, 1).unsqueeze(-1)  # [B, H, S, 1]

        # 加权融合
        attn_output = attn_output * gate_weights
        attn_output = attn_output.sum(dim=1)  # [B, S, D/H]
        attn_output = attn_output.reshape(batch_size, seq_len, self.d_model)

        # 输出投影
        return self.out_proj(attn_output)

# ---------- 使用示例 ----------
if __name__ == "__main__":
    d_model = 512
    num_heads = 8
    seq_len = 128
    batch_size = 4

    mla = MultiHeadLatentAttention(d_model, num_heads, latent_rank=32)
    x = torch.randn(batch_size, seq_len, d_model)
    output = mla(x)
    print(output.shape)  # [4, 128, 512]

关键代码解析‌

低秩压缩‌（对应原理2.1）：

键（U_k和V_k）与值（U_v和V_v）的投影被分解为两个低秩矩阵，例如：
python
Copy Code
k_latent = self.U_k(x)  # 第一次低秩投影
k = self.V_k(k_latent)  # 第二次恢复维度


潜在偏置‌（对应原理2.2）：

python
Copy Code
self.latent_bias = nn.Parameter(torch.zeros(num_heads, 1, 1))  # 每个头独立偏置
attn_scores = attn_scores + self.latent_bias  # 动态调整注意力权重


动态头融合‌（对应原理2.3）：

python
Copy Code
gate_weights = F.softmax(self.gate(x), dim=-1)  # 输入自适应的门控权重
attn_output = attn_output * gate_weights  # 按头加权

性能优化技巧‌
缓存中间结果‌：在推理时缓存 k_latent 和 v_latent，避免重复计算。
参数共享‌：可将 U_k 和 U_v 共享同一权重矩阵以减少参数量。
混合精度训练‌：使用 torch.cuda.amp 加速计算。

该代码实现了MLA的核心逻辑，可直接替换标准Transformer中的多头注意力模块。